In [1]:
import pandas as pd

In [25]:
# Load the CSV file
df = pd.read_csv("C:\\Users\\ABAIS\\OneDrive\\Desktop\\STEM\\Data_Eng_projects\\ETL_Projects\\ETL_TEXT_TO_CSV\\data.csv")

In [57]:
df.head()

,OrderID,CustomerID,ProductID,ProductName,CustomerName,Email,PhoneNumber,Quantity,Price,OrderDate,ShippingAddress,DateKey,AddressID,Customer_Name,Phone_Number
0,907237,5278,2291,case,Jared Willis,amcpherson@example.org,+1-925-909-0932x084,4,263.75,2023-12-10,"9274 White Islands\nNew Elizabethfurt, KY 02057",20231210,1,,
1,348169,7323,8972,around,John Sloan,williemccann@example.com,001-758-810-0709x58574,4,133.50,2024-01-04,"4168 Joshua Bridge Apt. 414\nNorth Crystal, OH...",20240104,2,,
2,172894,1193,5948,the,Kristin Ramos,ashleyhall@example.org,(478)406-7117x131,8,692.84,2024-02-02,"8508 Donald Inlet\nEast Paulchester, MH 97038",20240202,3,,
3,661569,7074,8505,or,Kathleen Johnson,alisonmitchell@example.net,655.589.5917x54016,4,596.57,2024-04-13,"05687 Perez Plaza\nPort Shannon, WI 60907",20240413,4,,
4,301661,9709,1353,three,Anthony Ramirez,ppitts@example.com,(800)355-1162x06927,6,70.15,2023-09-01,"33796 Herring Manor Suite 808\nHayesland, PW 5...",20230901,5,,


In [67]:

# Create Dimension Table: Customer_dim
customer_dim = df[['CustomerID', 'CustomerName', 'Email', 'PhoneNumber']].drop_duplicates()
customer_dim = customer_dim.reset_index(drop=True)
customer_dim.columns = ['CustomerID', 'CustomerName', 'Email', 'PhoneNumber']

In [55]:
customer_dim.head()

,CustomerID,CustomerName,Email,PhoneNumber
0,5278,Jared Willis,amcpherson@example.org,+1-925-909-0932x084
1,7323,John Sloan,williemccann@example.com,001-758-810-0709x58574
2,1193,Kristin Ramos,ashleyhall@example.org,(478)406-7117x131
3,7074,Kathleen Johnson,alisonmitchell@example.net,655.589.5917x54016
4,9709,Anthony Ramirez,ppitts@example.com,(800)355-1162x06927


In [69]:
# Create Dimension Table: Product
product_dim = df[['ProductID', 'ProductName']].drop_duplicates().copy()
customer_dim = customer_dim.reset_index(drop=True)
product_dim.columns = ['ProductID', 'ProductName']  # Renaming to match the first code logic
product_dim['Category'] = ''  # Placeholder for additional info
product_dim['Brand'] = ''

In [71]:
product_dim.head()

,ProductID,ProductName,Category,Brand
0,2291,case,,
1,8972,around,,
2,5948,the,,
3,8505,or,,
4,1353,three,,


In [73]:
# Create Dimension Table: Date
df['OrderDate'] = pd.to_datetime(df['OrderDate'])
date_dim = df[['OrderDate']].drop_duplicates().copy()
date_dim.columns = ['Order_Date']  # Renaming to match the first code logic
date_dim['Date_Key'] = date_dim['Order_Date'].dt.strftime('%Y%m%d')
date_dim['Day'] = date_dim['Order_Date'].dt.day
date_dim['Month'] = date_dim['Order_Date'].dt.month
date_dim['Year'] = date_dim['Order_Date'].dt.year
date_dim['Quarter'] = date_dim['Order_Date'].dt.to_period('Q').astype(str)


In [75]:
date_dim.head()

,Order_Date,Date_Key,Day,Month,Year,Quarter
0,2023-12-10,20231210,10,12,2023,2023Q4
1,2024-01-04,20240104,4,1,2024,2024Q1
2,2024-02-02,20240202,2,2,2024,2024Q1
3,2024-04-13,20240413,13,4,2024,2024Q2
4,2023-09-01,20230901,1,9,2023,2023Q3


In [85]:
# Create Dimension Table: Shipping Address
shipping_address_dim = df[['ShippingAddress']].drop_duplicates().copy()
shipping_address_dim.columns = ['ShippingAddress']  # Renaming to match the first code logic
shipping_address_dim['Address_ID'] = shipping_address_dim.index + 1


In [87]:
shipping_address_dim.head()

,ShippingAddress,Address_ID
0,"9274 White Islands\nNew Elizabethfurt, KY 02057",1
1,"4168 Joshua Bridge Apt. 414\nNorth Crystal, OH...",2
2,"8508 Donald Inlet\nEast Paulchester, MH 97038",3
3,"05687 Perez Plaza\nPort Shannon, WI 60907",4
4,"33796 Herring Manor Suite 808\nHayesland, PW 5...",5


In [89]:
# Create Fact Table: Sales
# Map the Date to DateKey
df = pd.merge(df, date_dim[['Order_Date', 'Date_Key']], left_on='OrderDate', right_on='Order_Date', how='left')

In [91]:
df.head()

,OrderID,CustomerID,ProductID,ProductName,CustomerName,Email,PhoneNumber,Quantity,Price,OrderDate,ShippingAddress,DateKey,AddressID,Customer_Name,Phone_Number,Order_Date,Date_Key
0,907237,5278,2291,case,Jared Willis,amcpherson@example.org,+1-925-909-0932x084,4,263.75,2023-12-10,"9274 White Islands\nNew Elizabethfurt, KY 02057",20231210,1,,,2023-12-10,20231210
1,348169,7323,8972,around,John Sloan,williemccann@example.com,001-758-810-0709x58574,4,133.50,2024-01-04,"4168 Joshua Bridge Apt. 414\nNorth Crystal, OH...",20240104,2,,,2024-01-04,20240104
2,172894,1193,5948,the,Kristin Ramos,ashleyhall@example.org,(478)406-7117x131,8,692.84,2024-02-02,"8508 Donald Inlet\nEast Paulchester, MH 97038",20240202,3,,,2024-02-02,20240202
3,661569,7074,8505,or,Kathleen Johnson,alisonmitchell@example.net,655.589.5917x54016,4,596.57,2024-04-13,"05687 Perez Plaza\nPort Shannon, WI 60907",20240413,4,,,2024-04-13,20240413
4,301661,9709,1353,three,Anthony Ramirez,ppitts@example.com,(800)355-1162x06927,6,70.15,2023-09-01,"33796 Herring Manor Suite 808\nHayesland, PW 5...",20230901,5,,,2023-09-01,20230901


In [115]:
# Map the Shipping Address to Address_ID
df = pd.merge(df, shipping_address_dim[['ShippingAddress', 'Address_ID']], left_on='ShippingAddress', right_on='ShippingAddress', how='left')

In [117]:
df.head()

,OrderID,CustomerID,ProductID,ProductName,CustomerName,Email,PhoneNumber,Quantity,Price,OrderDate,...,AddressID,Customer_Name,Phone_Number,Order_Date_x,Date_Key_x,Order_Date_y,Date_Key_y,Order_Date,Date_Key,Address_ID
0,907237,5278,2291,case,Jared Willis,amcpherson@example.org,+1-925-909-0932x084,4,263.75,2023-12-10,...,1,,,2023-12-10,20231210,2023-12-10,20231210,2023-12-10,20231210,1
1,348169,7323,8972,around,John Sloan,williemccann@example.com,001-758-810-0709x58574,4,133.50,2024-01-04,...,2,,,2024-01-04,20240104,2024-01-04,20240104,2024-01-04,20240104,2
2,172894,1193,5948,the,Kristin Ramos,ashleyhall@example.org,(478)406-7117x131,8,692.84,2024-02-02,...,3,,,2024-02-02,20240202,2024-02-02,20240202,2024-02-02,20240202,3
3,661569,7074,8505,or,Kathleen Johnson,alisonmitchell@example.net,655.589.5917x54016,4,596.57,2024-04-13,...,4,,,2024-04-13,20240413,2024-04-13,20240413,2024-04-13,20240413,4
4,301661,9709,1353,three,Anthony Ramirez,ppitts@example.com,(800)355-1162x06927,6,70.15,2023-09-01,...,5,,,2023-09-01,20230901,2023-09-01,20230901,2023-09-01,20230901,5


In [119]:
# Select columns for Fact Table
fact_table = df[['OrderID', 'CustomerID', 'ProductID', 'Quantity', 'Price', 'Date_Key', 'Address_ID']].copy()
fact_table.columns = ['Order_ID', 'Customer_ID', 'Product_ID', 'Quantity', 'Price', 'Date_Key', 'Address_ID']  # Renaming to match the first code logic


In [121]:
fact_table.head()

,Order_ID,Customer_ID,Product_ID,Quantity,Price,Date_Key,Address_ID
0,907237,5278,2291,4,263.75,20231210,1
1,348169,7323,8972,4,133.50,20240104,2
2,172894,1193,5948,8,692.84,20240202,3
3,661569,7074,8505,4,596.57,20240413,4
4,301661,9709,1353,6,70.15,20230901,5
